In [1]:
import os

In [2]:
%pwd

'c:\\Users\\NARINDER\\Desktop\\new project\\research'

In [3]:
os.chdir('..')

In [4]:
%pwd

'c:\\Users\\NARINDER\\Desktop\\new project'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class BaseModelConfig:
    root_dir: Path
    base_model_path: Path

In [6]:
import yaml
from pathlib import Path
from typing import Any, Dict
from src.floodClassifier.constants import *

class ConfigurationManager:
    def __init__(self, config_path = CONFIG_FILE_PATH, params_path = PARAMS_FILE_PATH):
        self.config_path = Path(config_path)
        self.params_path = Path(params_path)
        if not self.config_path.exists():
            raise FileNotFoundError(f"Config file not found: {self.config_path}")
        if not self.params_path.exists():
            raise FileNotFoundError(f"Params file not found: {self.params_path}")
        self._config = yaml.safe_load(self.config_path.read_text())
        self._params = yaml.safe_load(self.params_path.read_text())

    def get_prepare_base_model_config(self) -> Dict[str, Any]:
        pb = self._config.get("prepare_base_model", {})
        return {
            "root_dir": pb.get("root_dir", "artifacts/prepareBaseModel"),
            "base_model_path": pb.get("base_model_path", "artifacts/prepareBaseModel/base_model.pkl")
        }

    def get_arima_params(self) -> Dict[str, Any]:
        model_cfg = self._params.get("MODEL", {})
        tuning_cfg = self._params.get("TUNING", {})
        forecast_cfg = self._params.get("FORECAST", {})
        metrics = self._params.get("METRICS", [])
        return {
            "type": model_cfg.get("TYPE", "arima"),
            "p": int(model_cfg.get("P", 1)),
            "d": int(model_cfg.get("D", 0)),
            "q": int(model_cfg.get("Q", 0)),
            "seasonal": bool(model_cfg.get("SEASONAL", False)),
            "m": int(model_cfg.get("M", 0)),
            "enforce_stationarity": bool(model_cfg.get("ENFORCE_STATIONARITY", True)),
            "enforce_invertibility": bool(model_cfg.get("ENFORCE_INVERTIBILITY", True)),
            "tuning_enabled": bool(tuning_cfg.get("ENABLED", False)),
            "tuning_search": tuning_cfg.get("SEARCH", "grid"),
            "p_values": tuning_cfg.get("P_VALUES", []),
            "d_values": tuning_cfg.get("D_VALUES", []),
            "q_values": tuning_cfg.get("Q_VALUES", []),
            "horizon": int(forecast_cfg.get("HORIZON", 30)),
            "conf_int": float(forecast_cfg.get("CONF_INT", 0.95)),
            "metrics": metrics
        }

In [7]:
import os
import pickle
from pathlib import Path
import numpy as np
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX
from typing import Optional, Dict

class PrepareBaseModel:
    def __init__(self, prepare_cfg: Dict[str, str], arima_params: Dict[str, any]):
        self.root_dir = Path(prepare_cfg["root_dir"])
        self.base_model_path = Path(prepare_cfg["base_model_path"])
        self.arima_params = arima_params
        os.makedirs(self.root_dir, exist_ok=True)

    def _ensure_series(self, y: pd.Series) -> pd.Series:
        if not isinstance(y, pd.Series):
            y = pd.Series(y)
        if y.index.dtype == object:
            try:
                y.index = pd.to_datetime(y.index)
            except Exception:
                pass
        return y.astype(float)

    def build_arima(self, y: pd.Series, exog: Optional[pd.DataFrame] = None):
        y = self._ensure_series(y)
        p = self.arima_params["p"]
        d = self.arima_params["d"]
        q = self.arima_params["q"]
        seasonal = self.arima_params["seasonal"]
        m = self.arima_params["m"] if seasonal else 0

        model = SARIMAX(
            endog=y,
            exog=exog,
            order=(p, d, q),
            seasonal_order=(0, 0, 0, 0) if not seasonal else (p, d, q, m),
            enforce_stationarity=self.arima_params.get("enforce_stationarity", True),
            enforce_invertibility=self.arima_params.get("enforce_invertibility", True),
            simple_differencing=False
        )
        return model

    def fit_and_save(self, y: pd.Series, exog: Optional[pd.DataFrame] = None, save_overwrite: bool = True):
        model = self.build_arima(y, exog=exog)
        fitted = model.fit(disp=False)
        if self.base_model_path.exists() and not save_overwrite:
            raise FileExistsError(f"Base model already exists at {self.base_model_path}")
        with open(self.base_model_path, "wb") as f:
            pickle.dump(fitted, f)
        return fitted

    def load(self):
        if not self.base_model_path.exists():
            raise FileNotFoundError(f"No base model at {self.base_model_path}")
        with open(self.base_model_path, "rb") as f:
            return pickle.load(f)

    def run_from_df(self, df: pd.DataFrame, target_col: str = "y", exog_cols: Optional[list] = None):
        if target_col not in df.columns:
            raise KeyError(f"target_col '{target_col}' not found in dataframe")
        y = df[target_col]
        exog = df[exog_cols] if exog_cols else None
        fitted = self.fit_and_save(y, exog=exog)
        return fitted


In [9]:
from src.floodClassifier import logger

try:
    config = ConfigurationManager()
    prepare_cfg = config.get_prepare_base_model_config()
    arima_params = config.get_arima_params()
    preparer = PrepareBaseModel(prepare_cfg, arima_params)

    csv_path = r"artifacts\data_ingestion\FloodPrediction.csv"

    date_year_col = "Year"
    date_month_col = "Month"
    target_col = "Flood?"
    exog_cols = ["Rainfall", "Max_Temp", "Min_Temp", "Relative_Humidity"]
    usecols = [date_year_col, date_month_col, target_col] + exog_cols
    df = pd.read_csv(csv_path, usecols=usecols)

    try:
        df["Month"] = df[date_month_col].astype(int)
        df["Year"] = df[date_year_col].astype(int)
        df["_date"] = pd.to_datetime(dict(year=df["Year"], month=df["Month"], day=1))
    except Exception:
        df["_date"] = pd.to_datetime(df[date_year_col].astype(str) + "-" + df[date_month_col].astype(str) + "-01", errors="coerce")

    df = df.dropna(subset=["_date"]).copy()
    df = df.set_index("_date")
    df = df.sort_index()

    if target_col not in df.columns:
        raise KeyError(f"Target column '{target_col}' not found in file: {csv_path}")

    df = df.dropna(subset=[target_col])

    df[target_col] = pd.to_numeric(df[target_col], errors="coerce")
    if df[target_col].isna().any():
        raise ValueError("Target column contains non-numeric or uncoercible values after parsing")

    if exog_cols:
        for col in exog_cols:
            df[col] = pd.to_numeric(df[col], errors="coerce")
        df[exog_cols] = df[exog_cols].fillna(method="ffill").fillna(method="bfill")

    if not df.index.is_monotonic_increasing:
        df = df.sort_index()

    required_obs = max(arima_params.get("p", 1), arima_params.get("d", 0), arima_params.get("q", 1)) + 1
    if len(df) < required_obs:
        raise ValueError(f"Not enough observations ({len(df)}) to fit ARIMA with order p,d,q requiring at least {required_obs}")

    fitted_model = preparer.run_from_df(df, target_col=target_col, exog_cols=exog_cols)
    logger.info(f"ARIMA base model fitted and saved to: {prepare_cfg['base_model_path']}")

except Exception as e:
    raise e

C:\Users\NARINDER\AppData\Local\Temp\ipykernel_604\1717932242.py:41: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[exog_cols] = df[exog_cols].fillna(method="ffill").fillna(method="bfill")
c:\Users\NARINDER\Desktop\new project\.venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\NARINDER\Desktop\new project\.venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


[2025-11-21 02:55:22,635: INFO: 1717932242: ARIMA base model fitted and saved to: artifacts/prepareBaseModel/base_model.h5]


In [12]:
import pickle
from pathlib import Path
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error
from src.floodClassifier.constants import *

config = ConfigurationManager()
prepare_cfg = config.get_prepare_base_model_config()
arima_params = config.get_arima_params()

base_model_path = Path(prepare_cfg["base_model_path"])
csv_path = r"C:\Users\NARINDER\Desktop\new project\artifacts\data_ingestion\FloodPrediction.csv"

date_year_col = "Year"
date_month_col = "Month"
target_col = "Flood?"
exog_cols = ["Rainfall", "Max_Temp"]
test_fraction = 0.2
forecast_horizon = arima_params.get("horizon", 30)

df = pd.read_csv(csv_path, usecols=[date_year_col, date_month_col, target_col] + exog_cols)
df["Month"] = df[date_month_col].astype(int)
df["Year"] = df[date_year_col].astype(int)
df["_date"] = pd.to_datetime(dict(year=df["Year"], month=df["Month"], day=1))
df = df.dropna(subset=["_date"])
df = df.set_index("_date").sort_index()
df[target_col] = pd.to_numeric(df[target_col], errors="coerce")
df = df.dropna(subset=[target_col])
if exog_cols:
    for c in exog_cols:
        df[c] = pd.to_numeric(df[c], errors="coerce")
    df[exog_cols] = df[exog_cols].fillna(method="ffill").fillna(method="bfill")

n = len(df)
n_test = max(1, int(n * test_fraction))
train_df = df.iloc[:-n_test]
test_df = df.iloc[-n_test:]

y_train = train_df[target_col]
y_test = test_df[target_col]
exog_train = train_df[exog_cols] if exog_cols else None
exog_test = test_df[exog_cols] if exog_cols else None

if not base_model_path.exists():
    raise FileNotFoundError(f"Base model not found at {base_model_path}")

with open(base_model_path, "rb") as f:
    fitted = pickle.load(f)

steps = len(test_df)
try:
    fc = fitted.get_forecast(steps=steps, exog=exog_test)
    y_pred = fc.predicted_mean
    conf_int = fc.conf_int(alpha=1 - arima_params.get("conf_int", 0.95))
except Exception:
    start = test_df.index[0]
    end = test_df.index[-1]
    y_pred = fitted.predict(start=start, end=end, exog=exog_test)
    conf_int = None

import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error

try:
    y_pred = pd.Series(y_pred)
except Exception:
    y_pred = pd.Series(list(y_pred))

print("DEBUG pre-align lengths -> y_test:", len(y_test), "y_pred:", len(y_pred))
try:
    print("DEBUG pre-align index ranges -> y_test:", y_test.index.min(), "to", y_test.index.max())
except Exception:
    print("DEBUG y_test has no index")
try:
    print("DEBUG y_pred index sample:", getattr(y_pred, "index", None)[:3], "len:", len(y_pred))
except Exception:
    pass

try:
    y_pred_idx = pd.Series(y_pred.values, index=test_df.index)
except Exception:
    y_pred_idx = pd.Series(y_pred.values[:len(test_df)], index=test_df.index[:len(y_pred.values[:len(test_df)])])

y_test_s = y_test.astype(float)

combined = pd.concat([y_test_s, y_pred_idx], axis=1)
combined.columns = ["y_true", "y_pred"]
combined = combined.dropna(how="any")

if combined.empty:
    print("DEBUG: index alignment produced empty combined. Trying positional fallback.")
    min_len = min(len(y_test_s), len(y_pred))
    if min_len > 0:
        y_true_pos = y_test_s.values[-min_len:].astype(float)
        y_pred_pos = y_pred.values[-min_len:].astype(float)
        combined = pd.DataFrame({"y_true": y_true_pos, "y_pred": y_pred_pos})
    else:
        combined = pd.DataFrame()

if combined.empty:
    print("DEBUG: positional fallback produced empty. Trying element-wise non-NaN overlap.")
    a = y_test_s.values
    b = y_pred.values
    min_len = min(len(a), len(b))
    if min_len > 0:
        a = a[-min_len:]
        b = b[-min_len:]
        mask = np.isfinite(a) & np.isfinite(b)
        if mask.any():
            combined = pd.DataFrame({"y_true": a[mask], "y_pred": b[mask]})
        else:
            combined = pd.DataFrame()

if combined.empty:
    print("DEBUG: final diagnostics -> len(y_test):", len(y_test), "len(y_pred):", len(y_pred))
    print("DEBUG sample y_test tail:", y_test.tail(5))
    print("DEBUG sample y_pred tail:", pd.Series(y_pred).tail(5))
    raise ValueError("No overlapping non-NaN samples available for evaluation after all fallbacks")

y_true = combined["y_true"].astype(float).values
y_pred_vals = combined["y_pred"].astype(float).values

mae = mean_absolute_error(y_true, y_pred_vals)
rmse = mean_squared_error(y_true, y_pred_vals)

print(f"Evaluated samples: {len(y_true)}")
print(f"MAE: {mae:.4f}")
print(f"RMSE: {rmse:.4f}")

# future_steps = min(forecast_horizon, 12)
# if exog_cols:
#     last_exog = df[exog_cols].iloc[-1:]
#     future_exog = pd.concat([last_exog]*future_steps, ignore_index=True)
#     future_index = pd.date_range(start=df.index[-1] + pd.offsets.MonthBegin(1), periods=future_steps, freq="MS")
#     future_exog.index = future_index
# else:
#     future_exog = None
#     future_index = pd.date_range(start=df.index[-1] + pd.offsets.MonthBegin(1), periods=future_steps, freq="MS")

# try:
#     future_fc = fitted.get_forecast(steps=future_steps, exog=future_exog)
#     future_pred = pd.Series(future_fc.predicted_mean.values, index=future_index)
#     future_ci = future_fc.conf_int(alpha=1 - arima_params.get("conf_int", 0.95))
# except Exception:
#     future_pred = pd.Series(fitted.predict(start=future_index[0], end=future_index[-1], exog=future_exog),
#                             index=future_index)
#     future_ci = None

# print("\nFuture predictions (quick):")
# print(future_pred)

DEBUG pre-align lengths -> y_test: 898 y_pred: 915
DEBUG pre-align index ranges -> y_test: 2004-07-01 00:00:00 to 2013-12-01 00:00:00
DEBUG y_pred index sample: DatetimeIndex(['2004-07-01', '2004-07-01', '2004-07-01'], dtype='datetime64[ns]', name='_date', freq=None) len: 915
Evaluated samples: 898
MAE: 0.2438
RMSE: 0.1965


C:\Users\NARINDER\AppData\Local\Temp\ipykernel_604\4154752488.py:33: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[exog_cols] = df[exog_cols].fillna(method="ffill").fillna(method="bfill")
c:\Users\NARINDER\Desktop\new project\.venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\NARINDER\Desktop\new project\.venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


In [13]:
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix


# prob = np.clip(y_pred, 0.0, 1.0)
# pred_label = (prob >= 0.5).astype(int)
# true_label = y_test.astype(int)

# print("accuracy", accuracy_score(true_label, pred_label))
# print("precision", precision_score(true_label, pred_label, zero_division=0))
# print("recall", recall_score(true_label, pred_label, zero_division=0))
# print("f1", f1_score(true_label, pred_label, zero_division=0))
# print("confusion:\n", confusion_matrix(true_label, pred_label))